In [2]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, \
LogisticRegression
import duckdb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import statsmodels.api as sm
from scipy import stats 
from scipy.stats import permutation_test

In [3]:
#import packages and read csv file
raw_df=pd.read_csv("/Users/ashleychiang/2950PROJECT/\
Amazon Customer Behavior Survey.csv",thousands=",")

print(raw_df.shape)
raw_df.head()

(602, 23)


,Timestamp,age,Gender,Purchase_Frequency,Purchase_Categories,Personalized_Recommendation_Frequency,Browsing_Frequency,Product_Search_Method,Search_Result_Exploration,Customer_Reviews_Importance,...,Saveforlater_Frequency,Review_Left,Review_Reliability,Review_Helpfulness,Personalized_Recommendation_Frequency,Recommendation_Helpfulness,Rating_Accuracy,Shopping_Satisfaction,Service_Appreciation,Improvement_Areas
0,2023/06/04 1:28:19 PM GMT+5:30,23,Female,Few times a month,Beauty and Personal Care,Yes,Few times a week,Keyword,Multiple pages,1,...,Sometimes,Yes,Occasionally,Yes,2,Yes,1,1,Competitive prices,Reducing packaging waste
1,2023/06/04 2:30:44 PM GMT+5:30,23,Female,Once a month,Clothing and Fashion,Yes,Few times a month,Keyword,Multiple pages,1,...,Rarely,No,Heavily,Yes,2,Sometimes,3,2,Wide product selection,Reducing packaging waste
2,2023/06/04 5:04:56 PM GMT+5:30,24,Prefer not to say,Few times a month,Groceries and Gourmet Food;Clothing and Fashion,No,Few times a month,Keyword,Multiple pages,2,...,Rarely,No,Occasionally,No,4,No,3,3,Competitive prices,Product quality and accuracy
3,2023/06/04 5:13:00 PM GMT+5:30,24,Female,Once a month,Beauty and Personal Care;Clothing and Fashion;...,Sometimes,Few times a month,Keyword,First page,5,...,Sometimes,Yes,Heavily,Yes,3,Sometimes,3,4,Competitive prices,Product quality and accuracy
4,2023/06/04 5:28:06 PM GMT+5:30,22,Female,Less than once a month,Beauty and Personal Care;Clothing and Fashion,Yes,Few times a month,Filter,Multiple pages,1,...,Rarely,No,Heavily,Yes,4,Yes,2,2,Competitive prices,Product quality and accuracy


## Data Cleaning: 
Based on our research questions and our interests, we filtered out 6 variables/columns that are relevant toward our study, including age, review helpfulness, recommendation helpfulness, personalized recommendation frequency, cart completion frequency, add to cart browsing, and customer reviews importance. 

In [11]:
#we created a new dataframe called ini_df which includes 
#all the relevant variables to our research question.

ini_df= duckdb.sql('''

SELECT age, Review_Helpfulness, Recommendation_Helpfulness, 
Personalized_Recommendation_Frequency, Cart_Completion_Frequency, 
Add_to_Cart_Browsing
FROM raw_df

''').df()

ini_df.head()

,age,Review_Helpfulness,Recommendation_Helpfulness,Personalized_Recommendation_Frequency,Cart_Completion_Frequency,Add_to_Cart_Browsing
0,23,Yes,Yes,Yes,Sometimes,Yes
1,23,Yes,Sometimes,Yes,Often,Yes
2,24,No,No,No,Sometimes,Yes
3,24,Yes,Sometimes,Sometimes,Sometimes,Maybe
4,22,Yes,Yes,Yes,Sometimes,Yes


Since some of the columns are strings (i.e. Yes, Sometimes, No) and cannot be numerically measured, we change those strings to corresponding numbers (i.e. 0, 1, 2) with new columns, including columns such as review helpfulness, recommendation helpfulness, personalized recommendation frequency, cart completion frequency, add to cart browsing, and customer reviews importance. 

In [6]:
#We changed the values from string to numbers so 
#that we can actually measure the correlation. 


ini_df['New_Review_Helpfulness']=ini_df['Review_Helpfulness'].\
map({'No': 0, 'Sometimes': 1, 'Yes': 2})

ini_df['New_Recommendation_Helpfulness']=\
ini_df['Recommendation_Helpfulness'].\
map({'No': 0, 'Sometimes': 1, 'Yes': 2})

ini_df['New_Personalized_Recommendation_Frequency']=\
ini_df['Personalized_Recommendation_Frequency'].\
map({'No': 0, 'Sometimes': 1, 'Yes': 2})

ini_df['New_Cart_Completion_Frequency']=\
ini_df['Cart_Completion_Frequency'].\
map({'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often':3, 'Always':4})

ini_df['New_Add_to_Cart_Browsing']=ini_df['Add_to_Cart_Browsing'].\
map({'No': 0, 'Maybe': 1, 'Yes': 2})
ini_df.head()


,age,Review_Helpfulness,Recommendation_Helpfulness,Personalized_Recommendation_Frequency,Cart_Completion_Frequency,Add_to_Cart_Browsing,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing
0,23,Yes,Yes,Yes,Sometimes,Yes,2,2,2,2,2
1,23,Yes,Sometimes,Yes,Often,Yes,2,1,2,3,2
2,24,No,No,No,Sometimes,Yes,0,0,0,2,2
3,24,Yes,Sometimes,Sometimes,Sometimes,Maybe,2,1,1,2,1
4,22,Yes,Yes,Yes,Sometimes,Yes,2,2,2,2,2


We created a new dataframe called num_df by selecting the new columns and age as we will not be using original columns with strings in our project.

In [7]:
#We filtered out the old columns that weren't relevant 
#anymore into a new dataframe as we have updated the columns. 


num_df=duckdb.sql('''
SELECT age, New_Review_Helpfulness, New_Recommendation_Helpfulness, 
New_Personalized_Recommendation_Frequency, New_Cart_Completion_Frequency, 
New_Add_to_Cart_Browsing
FROM ini_df
'''                  
).df()
num_df.head()

,age,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing
0,23,2,2,2,2,2
1,23,2,1,2,3,2
2,24,0,0,0,2,2
3,24,2,1,1,2,1
4,22,2,2,2,2,2


In case of outliers and oddities with age, we filtered out the ages that are below 13 since they are not allowed to have an Amazon account. 

In [8]:
#We looked for outliers for age. We know that if age is under 13, 
#they can't make accounts to buy things. 

cleaned_df=duckdb.sql('''
SELECT *
FROM num_df
WHERE age>=13
''').df()

print(cleaned_df.shape)

cleaned_df.head()

(600, 6)


,age,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing
0,23,2,2,2,2,2
1,23,2,1,2,3,2
2,24,0,0,0,2,2
3,24,2,1,1,2,1
4,22,2,2,2,2,2


We categorized the ages into 5 distinct age groups which are 13-18, 19-25, 26-40, 41-64, and 65+ into a new dataframe called final_df with a new column called age_group.

In [9]:
#We want to categorize into 5 different age groups with the cleaned_df.

cleaned_df.loc[(cleaned_df["age"] >= 13) & (cleaned_df["age"] <= 18)\
               , "age_group"] = 1
cleaned_df.loc[(cleaned_df["age"] >= 19) & (cleaned_df["age"] <= 25)\
               , "age_group"] = 2
cleaned_df.loc[(cleaned_df["age"] >= 26) & (cleaned_df["age"] <= 40)\
               , "age_group"] = 3
cleaned_df.loc[(cleaned_df["age"] >= 41) & (cleaned_df["age"] <= 64)\
               , "age_group"] = 4
cleaned_df.loc[(cleaned_df["age"] >= 65), "age_group"] = 5

#We changed the type of age_group column from float to int. 
cleaned_df["age_group"] = cleaned_df["age_group"].astype(int)

final_df = cleaned_df

final_df.head()

,age,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing,age_group
0,23,2,2,2,2,2,2
1,23,2,1,2,3,2,2
2,24,0,0,0,2,2,2
3,24,2,1,1,2,1,2
4,22,2,2,2,2,2,2


We exported the cleaned dataset to a csv file.

In [10]:
final_df.to_csv('appendix.csv')